In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [5]:
iris_data = load_iris()

print('Example Data:')
print(iris_data.data[:5])
print('Example Labels:')
print(iris_data.target[:5])

Example Data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Example Labels:
[0 0 0 0 0]


In [52]:
x = iris_data.data
y_ = iris_data.target.reshape(-1,1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)

In [53]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.20)

In [54]:
model = Sequential()

model.add(Dense(10, input_shape=(4,), activation='relu', name='fc1'))
model.add(Dense(10, activation='relu', name='fc2'))
model.add(Dense(3, activation='softmax', name='output'))

In [55]:
optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [56]:
print('Neural Net Model Summary: ')
print(model.summary())

Neural Net Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 10)                50        
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
output (Dense)               (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
model.fit(train_x, train_y, verbose = 2, batch_size = 5, epochs = 200)

Epoch 1/200
 - 0s - loss: 1.8317 - acc: 0.2333
Epoch 2/200
 - 0s - loss: 1.3685 - acc: 0.4167
Epoch 3/200
 - 0s - loss: 1.1298 - acc: 0.6417
Epoch 4/200
 - 0s - loss: 1.0112 - acc: 0.6167
Epoch 5/200
 - 0s - loss: 0.9427 - acc: 0.5167
Epoch 6/200
 - 0s - loss: 0.8940 - acc: 0.6417
Epoch 7/200
 - 0s - loss: 0.8556 - acc: 0.6417
Epoch 8/200
 - 0s - loss: 0.8252 - acc: 0.5833
Epoch 9/200
 - 0s - loss: 0.7856 - acc: 0.6833
Epoch 10/200
 - 0s - loss: 0.7543 - acc: 0.6583
Epoch 11/200
 - 0s - loss: 0.7212 - acc: 0.6750
Epoch 12/200
 - 0s - loss: 0.6950 - acc: 0.6250
Epoch 13/200
 - 0s - loss: 0.6592 - acc: 0.6833
Epoch 14/200
 - 0s - loss: 0.6432 - acc: 0.6833
Epoch 15/200
 - 0s - loss: 0.6257 - acc: 0.5417
Epoch 16/200
 - 0s - loss: 0.5974 - acc: 0.6833
Epoch 17/200
 - 0s - loss: 0.5713 - acc: 0.6833
Epoch 18/200
 - 0s - loss: 0.5540 - acc: 0.6833
Epoch 19/200
 - 0s - loss: 0.5410 - acc: 0.6833
Epoch 20/200
 - 0s - loss: 0.5252 - acc: 0.6833
Epoch 21/200
 - 0s - loss: 0.5175 - acc: 0.6833
E

Epoch 171/200
 - 0s - loss: 0.1089 - acc: 0.9500
Epoch 172/200
 - 0s - loss: 0.1039 - acc: 0.9583
Epoch 173/200
 - 0s - loss: 0.1056 - acc: 0.9500
Epoch 174/200
 - 0s - loss: 0.1199 - acc: 0.9500
Epoch 175/200
 - 0s - loss: 0.1028 - acc: 0.9500
Epoch 176/200
 - 0s - loss: 0.1057 - acc: 0.9500
Epoch 177/200
 - 0s - loss: 0.1100 - acc: 0.9583
Epoch 178/200
 - 0s - loss: 0.1055 - acc: 0.9583
Epoch 179/200
 - 0s - loss: 0.1062 - acc: 0.9583
Epoch 180/200
 - 0s - loss: 0.1237 - acc: 0.9417
Epoch 181/200
 - 0s - loss: 0.1069 - acc: 0.9500
Epoch 182/200
 - 0s - loss: 0.1106 - acc: 0.9500
Epoch 183/200
 - 0s - loss: 0.1060 - acc: 0.9583
Epoch 184/200
 - 0s - loss: 0.1050 - acc: 0.9500
Epoch 185/200
 - 0s - loss: 0.1030 - acc: 0.9583
Epoch 186/200
 - 0s - loss: 0.1027 - acc: 0.9583
Epoch 187/200
 - 0s - loss: 0.1009 - acc: 0.9583
Epoch 188/200
 - 0s - loss: 0.1008 - acc: 0.9500
Epoch 189/200
 - 0s - loss: 0.1052 - acc: 0.9583
Epoch 190/200
 - 0s - loss: 0.1012 - acc: 0.9583
Epoch 191/200
 - 0s 

In [59]:
results = model.evaluate(test_x, test_y)

30/30 [==============================] - 0s 40us/step


In [60]:
print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

Final test set loss: 0.069307
Final test set accuracy: 0.966667


In [61]:
question = np.matrix([[5.1,3.5,1.4,0.2]])
print(question)
model.predict(question)

[[5.1 3.5 1.4 0.2]]


array([[9.9984813e-01, 1.5181182e-04, 2.9333090e-14]], dtype=float32)